# 模型的基线实现

- <a href='#random_guess'>基线模型I：random guesss</a>
- <a href='#tf-idf'>基线模型II：TF-IDF</a>

----

# 数据准备

In [5]:
# -*- coding:utf-8 -*-

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [7]:
# Load Data 
train_df = pd.read_csv('./ubuntu_dataset/train.csv')
test_df = pd.read_csv('./ubuntu_dataset/test.csv')
validataion_df = pd.read_csv('./ubuntu_dataset/valid.csv')

In [8]:
# 预备predict放置的向量
y_test = np.zeros(len(test_df))

In [9]:
train_df.head()

,Context,Utterance,Label
0,i think we could import the old comments via r...,basically each xfree86 upload will NOT force u...,1.0
1,I'm not suggesting all - only the ones you mod...,sorry __eou__ i thought it was ubuntu related....,0.0
2,afternoon all __eou__ not entirely related to ...,"Yep. __eou__ oh, okay. I wondered what happene...",0.0
3,interesting __eou__ grub-install worked with /...,thats the one __eou__,1.0
4,and because Python gives Mark a woody __eou__ ...,(i thought someone was going to make a joke ab...,1.0


In [10]:
train_df.describe()

,Label
count,1000000.000000
mean,0.499873
std,0.500000
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [11]:
test_df.head()

,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
0,anyone knows why my stock oneiric exports env ...,nice thanks! __eou__,"wrong channel for it, but check efnet.org, uno...","every time the kernel changes, you will lose v...",ok __eou__,!nomodeset > acer __eou__ I'm assuming it is a...,http://www.ubuntu.com/project/about-ubuntu/der...,thx __eou__ unfortunately the program isn't in...,how can I check? By doing a recovery for testi...,my humble apologies __eou__,#ubuntu-offtopic __eou__
1,i set up my hd such that i have to type a pass...,"so you dont know, ok, anyone else? __eou__ you...","nmap is nice, but it wasn't what I was looking...",ok __eou__,cdrom worked fine on windows. __eou__ i dont ...,"ah yes, i have read return as rerun __eou__",hm? __eou__,"not the case, LTS is every other .04 release. ...",Pretty much __eou__,I used the one I downloaded from AMD __eou__,"ffmpeg is part of the package , quixotedon , a..."
2,im trying to use ubuntu on my macbook pro reti...,just wondering how it runs __eou__,"yes, that's what I did, exported it to a ""id_d...",nothing - i am talking about the question of m...,that should fix the fonts being too large __eou__,"okay, so hcitool echos back hci0 <mac address ...",I get to the menu with options such as 'try ub...,why do u need analyzer __eou__ it is a toy __e...,Cntrl-C may stop the command but it doesn't fi...,"if you're only going to run Ubuntu, just get a...",the ones which are not picked up at the moment...
3,no suggestions? __eou__ links? __eou__ how can...,you cant load anything via usb or cd when luks...,-p sorry... __eou__ nmap -p22 __eou__ It d...,i guess so i can't even launch it. __eou__,noted __eou__,rxvt-unicode is one __eou__,I tarred all of ~ __eou__,I tarred all of ~ __eou__,"I don't really know if I can help, but I was c...","that works just fine, thanks! __eou__",thank you __eou__
4,I just added a second usb printer but not sure...,i was setting it up under the printer configur...,i'd say the most commonly venue would be via L...,"the old hardy man page, http://manpages.ubuntu...",i'll give a try __eou__,"by the way, the url you posted for davfs is fr...",http://ubuntuforums.org/showthread.php?t=15498...,"So I load up putty gui, then what do I do? __e...","you should read error messages, it says 'are ...",waiting the college semester to close just to ...,I was calling myself a jerk. All I know is tha...


In [12]:
test_df.describe()

,Context,Ground Truth Utterance,Distractor_0,Distractor_1,Distractor_2,Distractor_3,Distractor_4,Distractor_5,Distractor_6,Distractor_7,Distractor_8
count,18920,18920,18920,18920,18920,18920,18920,18920,18920,18920,18920
unique,18920,18026,14066,13998,14162,14116,14200,14149,14064,14068,14202
top,best usenet downloader? __eou__ __eot__ there ...,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__,thanks __eou__
freq,1,152,133,142,163,148,130,164,155,151,156


# <a name='random_guess'>基线模型I：random guesss</a>

In [13]:
def evaluate_recall(y, y_test, k=1):
    '''
    手动实现的recall@k的评估方法，评估前k个的召回率。
    '''
    num_examples = float(len(y))
    num_correct = 0
    
    for predictions,label in zip(y, y_test):
        if label in predictions[:k]:
            num_correct += 1
    return num_correct/num_examples

In [14]:
def predict_random(context, utterances):
    '''
    随机猜答案
    '''
    return np.random.choice(len(utterances), 10, replace=False) 

np.random.choice(dataset, size, replace=False, p =[])
- replace=False无放回
- replace=True 有放回，生成的结果可能有重复的
- p是指定概率分布，没有指定则是均匀分布

In [35]:
predict_random(test_df.Context[0], test_df.iloc[0,1:].values)

array([1, 0, 4, 6, 7, 2, 8, 5, 9, 3])

In [34]:
# Evaluate Random predictor
y_random = [predict_random(test_df.Context[x], test_df.iloc[x,1:].values) for x in range(len(test_df))]
for n in [1, 2, 5, 10]:
    print("Recall @ ({}, 10): {:g}".format(n, evaluate_recall(y_random, y_test, n)))

Recall @ (1, 10): 0.102537
Recall @ (2, 10): 0.204704
Recall @ (5, 10): 0.501163
Recall @ (10, 10): 1


---

# <a name='tf-idf'>基线模型II：TF-IDF</a>

In [48]:
class TFIDFPredictor(object):
    def __init__(self):
        self.vectorizer = TfidfVectorizer()
    
    def train(self, data):
        '''
        给定训练数据用于生成tf-idf词库
        '''
        # np.append(arr1, arr2) 将两个100w的向量拼接为（200w,)的向量
        self.vectorizer.fit(np.append(data.Context.values, data.Utterance.values))
    
    def predict(self, context, utterances):
        # Convert context and utterances into tfidf vector
        vector_context = self.vectorizer.transform([context])
        vector_doc = self.vectorizer.transform(utterances)
        
        # The dot product measures the similarity of the resulting vectors
        result = np.dot(vector_doc, vector_context.T).todense()
        result = np.asarray(result).flatten()
        
        # Sort by top results and return the indices in descending order
        return np.argsort(result, axis=0)[::-1]

In [52]:
# Evaluate TFIDF predictor
pred = TFIDFPredictor()
pred.train(train_df)
y = [pred.predict(test_df.Context[x], test_df.iloc[x,1:].values) for x in range(len(test_df))]
for n in [1, 2, 5, 10]:
    print("Recall @ ({}, 10): {:g}".format(n, evaluate_recall(y, y_test, n)))

Recall @ (1, 10): 0.485624
Recall @ (2, 10): 0.586681
Recall @ (5, 10): 0.762474
Recall @ (10, 10): 1


In [67]:
print('下面是步骤解析'.center(70,'*'))

*******************************下面是步骤解析********************************


In [61]:
# 解析步骤
Vec = TfidfVectorizer()
train_df.Context.values
train_df.Utterance.values
Vec.fit(np.append(train_df.Context.values,train_df.Utterance.values))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [62]:
# predict解析
# 将context问题在tf-idf词库对应的向量化
vector_context = Vec.transform([test_df.Context[0]])
# 将10个答案在tf-idf词库对应的向量化
vector_doc = Vec.transform(test_df.iloc[0,1:].values)

In [73]:
vector_context.toarray()
vector_doc.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [74]:
# 通过向量点积的方式，计算相似度
result = np.dot(vector_doc, vector_context.T).todense()

In [75]:
result

matrix([[0.04880734],
        [0.0348111 ],
        [0.02543456],
        [0.02057558],
        [0.03942579],
        [0.00243546],
        [0.06546679],
        [0.0499886 ],
        [0.02529586],
        [0.01088604]])

In [79]:
np.asarray(result).flatten() # 水平拉直为一维的向量 (10,)

array([0.04880734, 0.0348111 , 0.02543456, 0.02057558, 0.03942579,
       0.00243546, 0.06546679, 0.0499886 , 0.02529586, 0.01088604])

In [80]:
# 通过np.argsort排序
np.argsort(result, axis=0)[::-1]

matrix([[6],
        [7],
        [0],
        [4],
        [1],
        [2],
        [8],
        [3],
        [9],
        [5]])